In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as patches
import pandas as pd
import pprint
from mpl_toolkits.mplot3d import Axes3D
import matplotlib as mpl
import os
import csv
import json
import math
import seaborn as sns
from scipy.ndimage.filters import gaussian_filter
from scipy.stats import entropy
from os import listdir
from os.path import isfile, join, isdir

# HRV (using https://github.com/rhenanbartels/hrv)
from hrv.classical import frequency_domain
from hrv.rri import RRi
import peakutils

mpl.rcParams['figure.dpi'] = 144
mpl.rcParams['figure.figsize'] = (8, 5)

pp = pprint.PrettyPrinter(indent=4)

In [2]:
DATA_DIR = "/Users/jeremygordon/Google Drive/Academic/# UC Berkeley ISchool PhD/Research/# Current/Covert Embodied Choice/Data"
DATAFRAME_DIR = "/Users/jeremygordon/Google Drive/Academic/# UC Berkeley ISchool PhD/Research/# Current/Covert Embodied Choice/DataFrames"

### Script to compute RRI/HRV

In [6]:
def bvpPeaks(signal):
    cb = np.array(signal)
    x = peakutils.indexes(cb, thres=0.02/max(cb), min_dist=0.1)
    y = []
    i = 0
    while (i < (len(x)-1)):
        if x[i+1] - x[i] < 15:
            y.append(x[i])
            x = np.delete(x, i+1)
        else:
            y.append(x[i])
        i += 1
    return y

def getRRI(signal, start, sample_rate):
    peakIDX = bvpPeaks(signal)
    spr = 1 / sample_rate # seconds between readings
    start_time = float(start)
    timestamp = [start_time, (peakIDX[0] * spr) + start_time ] 
    ibi = [0, 0]
    for i in range(1, len(peakIDX)):
        timestamp.append(peakIDX[i] * spr + start_time)
        ibi.append((peakIDX[i] - peakIDX[i-1]) * spr)

    df = pd.DataFrame({'Timestamp': timestamp, 'IBI': ibi})
    return df

def getHRV(data, avg_heart_rate):
    rri = np.array(data['IBI']) * 1000
    RR_list = rri.tolist()
    #RR_diff = []
    RR_sqdiff = []
    RR_diff_timestamp = []
    cnt = 2
    while (cnt < (len(RR_list)-1)): 
        #RR_diff.append(abs(RR_list[cnt+1] - RR_list[cnt])) 
        RR_sqdiff.append(math.pow(RR_list[cnt+1] - RR_list[cnt], 2)) 
        RR_diff_timestamp.append(data['Timestamp'][cnt])
        cnt += 1
    hrv_window_length = 10
    window_length_samples = int(hrv_window_length*(avg_heart_rate/60))
    #SDNN = []
    RMSSD = []
    index = 1
    for val in RR_sqdiff:
        if index < int(window_length_samples):
            #SDNNchunk = RR_diff[:index:]
            RMSSDchunk = RR_sqdiff[:index:]
        else:
            #SDNNchunk = RR_diff[(index-window_length_samples):index:]
            RMSSDchunk = RR_sqdiff[(index-window_length_samples):index:]
        #SDNN.append(np.std(SDNNchunk))
        RMSSD.append(math.sqrt(np.std(RMSSDchunk)))
        index += 1
    dt = np.dtype('Float64')
    #SDNN = np.array(SDNN, dtype=dt)
    RMSSD = np.array(RMSSD, dtype=dt)
    df = pd.DataFrame({'Timestamp': RR_diff_timestamp, 'HRV': RMSSD})
    return df

def calculte_hrv_rri(dir):
    try:
        BVP_DF = pd.read_csv('%s/BVP.csv' % dir)
        HR_DF = pd.read_csv('%s/HR.csv' % dir)
    except:
        print("Can't find BVP or HR in %s" % dir)
        return
    column = list(HR_DF)[0]
    temp = HR_DF.drop(0, axis = 0)
    HR = temp[column]
    HR = HR.tolist()

    column2 = list(BVP_DF)[0]
    sample_rate = BVP_DF[column2][0]
    temp = BVP_DF.drop(0, axis = 0)
    temp['spData'] = 0
    temp.loc[temp[column2] > 0, 'spData'] = temp[column2]
    signal = temp['spData'].tolist()

    RRI_DF = getRRI(signal, column2, sample_rate)
    HRV_DF = getHRV(RRI_DF, np.mean(HR))

    HRV_DF.to_csv('%s/HRV.csv' % dir, index=False)
    RRI_DF.to_csv('%s/RRI.csv' % dir, index=False)

    print('Done, saved as: HRV.csv')

def create_hrv_rri_files():
    for dir in [x[0] for x in os.walk(DATA_DIR)]:
        if dir != DATA_DIR:
            rri_fn = '%s/RRI.csv' % dir
            if os.path.exists(rri_fn):
                print("Already have RRI")
            else:
                calculte_hrv_rri(dir)
            
create_hrv_rri_files()

Already have RRI
Already have RRI
Already have RRI
Already have RRI
Already have RRI
Can't find BVP or HR in /Users/jeremygordon/Google Drive/Academic/# UC Berkeley ISchool PhD/Research/# Current/Covert Embodied Choice/Data/1580931438
Already have RRI
Already have RRI
Already have RRI


//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:55: DeprecationWarning: Numeric-style type codes are deprecated and will result in an error in the future.


Done, saved as: HRV.csv
Already have RRI
Already have RRI
Already have RRI
Done, saved as: HRV.csv
Done, saved as: HRV.csv
Already have RRI
Already have RRI
Already have RRI
Already have RRI
Already have RRI
Already have RRI
Done, saved as: HRV.csv
Already have RRI
Already have RRI
Done, saved as: HRV.csv
Done, saved as: HRV.csv
Done, saved as: HRV.csv
Already have RRI
Already have RRI
Already have RRI
Done, saved as: HRV.csv
Already have RRI
Done, saved as: HRV.csv
Done, saved as: HRV.csv
Already have RRI
Done, saved as: HRV.csv
Already have RRI
Already have RRI
Already have RRI
Already have RRI


In [7]:
def load_json(file):
    data = None
    try:
        with open(file) as f:
            data = json.load(f)
    except:
        pass
        # print("Can't find %s" % file)
    return data

def df_to_tuples(df):
    return [tuple(x) for x in df.values]    

def generate_subject_df():
    """
    Load participant list and qualtrics data then create subject_df
    """
    
    post_session_df = pd.read_csv(DATA_DIR + "/CEC Post-Session.csv")
    post_session_df['SONA_ID'] = post_session_df['SONA_ID'].astype(str)
    post_session_df = post_session_df.set_index(['SONA_ID'])
    # Drop non-data rows
    post_session_df.drop(post_session_df.index[[0, 1]], inplace=True)
    DROP_COLS = ["RecipientLastName", "RecipientFirstName", "RecipientEmail", "ExternalReference", "LocationLatitude",
                 "LocationLongitude", "DistributionChannel", "UserLanguage", "EndDate", "Status", "IPAddress", "Progress",
                 "Duration (in seconds)", "Finished", "RecordedDate"]
    post_session_df.drop(columns=DROP_COLS, inplace=True)
    subject_df = pd.read_csv(DATA_DIR + "/CEC Participant List.csv")
    subject_df.drop(columns=['progress'], inplace=True)
    subject_df['sona_id'] = subject_df['sona_id'].astype(str)
    subject_df = subject_df.set_index(['sona_id'])
    subject_df = subject_df.assign(
        hand_order='[]',
        condition='',
        left_handed=False, 
        total_points=np.nan, 
        total_points_possible=np.nan, 
        total_matches=np.nan, 
        ts_session_start=np.nan, 
        ts_adversary=np.nan, 
        ts_session_end=np.nan, 
        session_duration=np.nan
    )
    subject_df = subject_df.join(post_session_df)
    for sona_id, subj in subject_df.iterrows():
        if subj['drop']:
            subject_df.drop(sona_id, inplace=True)
            print("Dropping %s" % sona_id)
        else:
            session_id = subj['session_id']
            full_path = DATA_DIR + '/' + str(session_id)
            if isdir(full_path):
                for file in listdir(full_path):
                    # Skip all but metadata file
                    if file.endswith("meta.json"):
                        data = load_json(full_path + '/' + file)
                        del data['hand_specs']
                        data['hand_order'] = json.dumps(data['hand_order'])
                        data['session_duration'] = int(data['ts_session_end'] - data['ts_session_start'])
                        for key, val in data.items():
                            subject_df.at[sona_id, key] = val
            else:
                print("%s is not dir" % full_path)
    subject_df = process_e4_data(subject_df)
    subject_df.drop(['notes', 'data_issues'], 1, inplace=True)
    print("Generated subject dataframe with %d subjects" % len(subject_df))
    return subject_df

def process_e4_data(subject_df):
    """
    Append EDA/HR/HRV data to subject_df
    """
    for sona_id, row in subject_df.iterrows():
        # Two tuples (ts_start, ts_end) for non-adversary portion and adversary portion
        nadv_window = (row.ts_session_start, row.ts_adversary)
        adv_window = (row.ts_adversary, row.ts_session_end)
        windows = [nadv_window, adv_window]
        sections = ['nadv', 'adv']
        DATA_TYPES = ['eda', 'hr']
        for dt in DATA_TYPES:
            df_emp = pd.read_csv('%s/%s/%s.csv' % (DATA_DIR, row.session_id, dt.upper()), names=[dt])
            first_ts = float(df_emp.loc[0])
            hz = float(df_emp.loc[1])
            for window, section in zip(windows, sections):
                start_ts, end_ts = window
                start_index = int(2 + hz * int(start_ts - first_ts))
                end_index = int(2 + hz * int(end_ts - first_ts))
                cropped = df_emp[dt].iloc[start_index:end_index]
                # print("%s (%s): %s secs and %d rows (%s)" % (sona_id, dt, end_ts - start_ts, len(cropped), section))
                val_list = cropped.values.tolist()
                col = '%s_%s' % (dt, section)                    
                subject_df.at[sona_id, col] = json.dumps(val_list)
        # rri CSVs generated via https://github.com/Jegama/empaticaHRV
        CALC_HRV = False
        if CALC_HRV:
            try:
                df_rri = pd.read_csv('%s/%s/RRI.csv' % (DATA_DIR, row.session_id), names=['Timestamp', 'IBI'], dtype=float, header=0).drop(0)
            except:
                print("Missing RRI for %s" % sona_id)
                continue
            for window, section in zip(windows, sections):
                start_ts, end_ts = window 
                in_window = df_rri[(df_rri['Timestamp'] >= start_ts) & (df_rri['Timestamp'] <= end_ts)]
                rri = RRi(in_window['IBI'], time=in_window['Timestamp'])
                results = frequency_domain(rri,
                    fs=4.0,
                    method='welch',
                    interp_method='cubic',
                    detrend='linear'
                )
                print(results)
                for key, val in results.items():
                    col = 'hrv_%s_%s' % (key, section)
                    subject_df.at[sona_id, col] = val
    return subject_df
        
def generate_trial_and_tracking_dfs(subject_df, trial_df, tracking_df, fixation_df):
    """
    For each subject in passed df, load all subject's trial data and generate trial_df
    """
    if trial_df is None:
        trial_df = pd.DataFrame(columns=['subject_choice', 'trial_id', 'points', 'correct', 
                                         'avoided_prediction', 'ts_start', 'ts_selection', 
                                         'ts_end', 'hand', 'practice', 'choice_made',
                                         'with_adversary', 'duration'])
    if tracking_df is None:
        tracking_df = pd.DataFrame(columns=[
            'hmd_yaw', 'hmd_roll', 'hmd_pitch', 'hmd_x', 'hmd_y', 'hmd_z', 
            'ctr_yaw', 'ctr_roll', 'ctr_pitch', 'ctr_x', 'ctr_y', 'ctr_z', 
            'gaze_or_x', 'gaze_or_y', 'gaze_or_z', 'gaze_dir_x', 'gaze_dir_y', 
            'gaze_dir_z', 'gaze_tgt_x', 'gaze_tgt_y', 'gaze_tgt_z', 'gaze_conv_dist', 
            'blinking', 'ts'])
    if fixation_df is None:
        fixation_df = pd.DataFrame(columns=['objectName', 'start_ts', 'stop_ts', 'duration'])
    
    for sona_id, row in subject_df.iterrows():
        done = False
        trial = 1
        n_records = 0
        n_fixations = 0
        trial_rows = []
        tracking_rows = []
        fixation_rows = []
        while not done:
            key = "%s_%d" % (sona_id, trial)
            if key in trial_df.index:
                print("Skipping %s, already built data" % key)
                trial += 1
                continue
            data = load_json(DATA_DIR + "/%s/session_%s_trial_%d.json" % (row.session_id, row.session_id, trial))
            if data is not None:
                n_records += len(data['records'])
                n_fixations += len(data['fixations'])                
                for record in data['records']:
                    record_key = "%s_%.3f" % (sona_id, record['ts'])
                    record['subject'] = sona_id
                    record['trial'] = trial
                    tracking_rows.append(pd.Series(record, name=record_key))
                del data['records']
                for fixation in data['fixations']:
                    fix_key = "%s_%.3f" % (sona_id, fixation['start_ts'])
                    fixation['subject'] = sona_id
                    fixation['trial'] = trial
                    fixation_rows.append(pd.Series(fixation, name=fix_key))
                del data['fixations']
                data['duration'] = int(data.get('ts_end', 0) - data['ts_start'])
                data['subject'] = sona_id
                data['trial'] = trial
                trial_rows.append(pd.Series(data, name=key))
                trial += 1
            else:
                done = True
        if trial_rows:
            trial_df = trial_df.append(trial_rows)
        if tracking_rows:
            tracking_df = tracking_df.append(tracking_rows)
        if fixation_rows:
            fixation_df = fixation_df.append(fixation_rows)        
        print("Loaded %d trials, %d records, %d fixations for subject %s" % (trial - 1, n_records, n_fixations, sona_id))
    return trial_df, tracking_df, fixation_df

In [8]:
subject_df = generate_subject_df()

Dropping 48425
Dropping 47529
Dropping 48009
Dropping 47464
Generated subject dataframe with 36 subjects


In [ ]:
def try_load_pickle(path):
    df = None
    try:
        df = pd.read_pickle(path)
    except:
        print("No file at %s" % path)
    return df
    
trial_df = try_load_pickle(DATAFRAME_DIR + '/trial_df.pickle')
tracking_df = try_load_pickle(DATAFRAME_DIR + '/tracking_df.pickle')
fixation_df = try_load_pickle(DATAFRAME_DIR + '/fixation_df.pickle')

trial_df, tracking_df, fixation_df = generate_trial_and_tracking_dfs(subject_df, trial_df, tracking_df, fixation_df)

Skipping 48295_1, already built data
Skipping 48295_2, already built data
Skipping 48295_3, already built data
Skipping 48295_4, already built data
Skipping 48295_5, already built data
Skipping 48295_6, already built data
Skipping 48295_7, already built data
Skipping 48295_8, already built data
Skipping 48295_9, already built data
Skipping 48295_10, already built data
Skipping 48295_11, already built data
Skipping 48295_12, already built data
Skipping 48295_13, already built data
Skipping 48295_14, already built data
Skipping 48295_15, already built data
Skipping 48295_16, already built data
Skipping 48295_17, already built data
Skipping 48295_18, already built data
Skipping 48295_19, already built data
Skipping 48295_20, already built data
Skipping 48295_21, already built data
Skipping 48295_22, already built data
Skipping 48295_23, already built data
Skipping 48295_24, already built data
Skipping 48295_25, already built data
Skipping 48295_26, already built data
Skipping 48295_27, al

Loaded 47 trials, 54039 records, 2294 fixations for subject 47784
Loaded 47 trials, 57823 records, 1240 fixations for subject 41516
Loaded 47 trials, 53699 records, 1627 fixations for subject 43931


In [13]:
trial_df

,avoided_prediction,choice_made,correct,duration,hand,points,practice,subject,subject_choice,trial_id,ts_end,ts_selection,ts_start,with_adversary,trial
48295_1,False,True,True,24,"{'table': ['1d', '1s'], 'priv': ['1o', '1s'], ...",1,True,48295,0,1,1.581101e+09,1.581101e+09,1.581101e+09,False,1
48295_2,False,True,True,23,"{'table': ['2s', '2d'], 'priv': ['1o', '3s'], ...",1,True,48295,1,2,1.581101e+09,1.581101e+09,1.581101e+09,False,2
48295_3,False,True,True,14,"{'table': ['2s', '1s'], 'priv': ['3d', '1o'], ...",1,True,48295,0,3,1.581101e+09,1.581101e+09,1.581101e+09,False,3
48295_4,False,True,True,14,"{'table': ['3d', '3s'], 'priv': ['2d', '1o'], ...",1,True,48295,1,4,1.581101e+09,1.581101e+09,1.581101e+09,False,4
48295_5,False,True,True,15,"{'table': ['3o', '1o'], 'priv': ['2s', '1d'], ...",1,False,48295,0,5,1.581101e+09,1.581101e+09,1.581101e+09,False,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47948_43,True,True,True,12,"{'table': ['2d', '3s'], 'priv': ['3o', '3d'], ...",1,False,47948,1,43,1.582586e+09,1.582586e+09,1.582586e+09,True,43
47948_44,True,True,True,12,"{'table': ['3s', '2d'], 'priv': ['1d', '3d'], ...",1,False,47948,1,44,1.582586e+09,1.582586e+09,1.582586e+09,True,44
47948_45,True,True,True,11,"{'table': ['3s', '2d'], 'priv': ['3o', '1s'], ...",1,False,47948,1,45,1.582586e+09,1.582586e+09,1.582586e+09,True,45
47948_46,False,True,True,12,"{'table': ['3d', '2o'], 'priv': ['1d', '2d'], ...",0,False,47948,0,46,1.582586e+09,1.582586e+09,1.582586e+09,True,46


In [14]:
def infer_missing_choice_made():
    # trial_df['ts_start'].max()
    # > 1582314056.713696
    
    trial_df['choice_made'] = True
    LAST_MISSING_CHOICE_MADE = 1582314056.713696
    selection_duration = trial_df['ts_end'] - trial_df['ts_selection']
    # Infer non-choice (timeout)
    infer_timed_out = (trial_df['ts_start'] <= LAST_MISSING_CHOICE_MADE) & \
        (selection_duration > 2.985) & \
        (selection_duration < 3.015) & \
        (trial_df['correct'] == False)
    trial_df.loc[infer_timed_out, 'choice_made'] = False
    print("Inferring choice made..")
    print(trial_df.choice_made.value_counts())
    # Now that this has run, choice_made should be reliable in the pickle'd df
    
INFER = False
if INFER:
    infer_missing_choice_made()

In [15]:
trial_df.to_pickle(DATAFRAME_DIR + '/trial_df.pickle')
tracking_df.to_pickle(DATAFRAME_DIR + '/tracking_df.pickle')
fixation_df.to_pickle(DATAFRAME_DIR + '/fixation_df.pickle')
subject_df.to_pickle(DATAFRAME_DIR + '/subject_df.pickle')
print("Saved")

Saved
